## Extraindo dados do Censo Escolar

Nesse notebook extraímos os dados do censo escolar para os anos de 2012 a 2024

In [1]:
import pandas as pd
import requests
import os
from tqdm import tqdm
from zipfile import ZipFile

In [2]:
from config import RANGE_ANOS_ESTUDO, DATA_FOLDER

In [3]:
RANGE_ANOS_ESTUDO

[2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]

In [4]:
def solve_file_microdados_censo(ano:int) -> str:

    fname = f'microdados_censo_{ano}.zip'
    if not os.path.exists(DATA_FOLDER):
        os.mkdir(DATA_FOLDER)
    fpath = os.path.join(DATA_FOLDER, fname)

    return fpath


def download_microdados_censo(ano:int)->str:

    link_download_microdados_censo = f'https://download.inep.gov.br/dados_abertos/microdados_censo_escolar_{ano}.zip'
    link_ano = link_download_microdados_censo.format(ano=ano)
    fpath = solve_file_microdados_censo(ano)
    with requests.get(link_ano, stream=True) as r:
        r.raise_for_status()
        with open(fpath, 'wb') as f:
            content = r.content
            f.write(content)

In [5]:
for ano in tqdm(RANGE_ANOS_ESTUDO):
    if not os.path.exists(solve_file_microdados_censo(ano)):
        print(f'Downloading microdados do censo escolar {ano}...')
        download_microdados_censo(ano)
    else:
        print(f'Microdados do censo escolar {ano} já baixados.')

100%|██████████| 13/13 [00:00<00:00, 42040.06it/s]

Microdados do censo escolar 2012 já baixados.
Microdados do censo escolar 2013 já baixados.
Microdados do censo escolar 2014 já baixados.
Microdados do censo escolar 2015 já baixados.
Microdados do censo escolar 2016 já baixados.
Microdados do censo escolar 2017 já baixados.
Microdados do censo escolar 2018 já baixados.
Microdados do censo escolar 2019 já baixados.
Microdados do censo escolar 2020 já baixados.
Microdados do censo escolar 2021 já baixados.
Microdados do censo escolar 2022 já baixados.
Microdados do censo escolar 2023 já baixados.
Microdados do censo escolar 2024 já baixados.


In [6]:
def solve_csv_data_path_within_zip(ano:int) -> str:
    
    fname: str = f'microdados_ed_basica_{ano}.csv'
    path: str = os.path.join(f'microdados_ed_basica_{ano}', 'dados', fname)

    return path

def read_data(ano:int)->pd.DataFrame:
    fpath = solve_file_microdados_censo(ano)
    if not os.path.exists(fpath):
        raise FileNotFoundError(f'Arquivo {fpath} não encontrado. Baixe os dados primeiro.')

    zip_file_path = solve_csv_data_path_within_zip(ano)
    with ZipFile(fpath, 'r') as zip_ref:
        if zip_file_path not in zip_ref.namelist():
            raise FileNotFoundError(f'Arquivo {zip_file_path} não encontrado no zip {fpath}.')
        with zip_ref.open(zip_file_path) as file:
            df = pd.read_csv(file, sep=';', encoding='latin1', low_memory=False)
        return df

In [7]:
df = read_data(2012)

In [8]:
df.head()

,NU_ANO_CENSO,NO_REGIAO,CO_REGIAO,NO_UF,SG_UF,CO_UF,NO_MUNICIPIO,CO_MUNICIPIO,NO_MESORREGIAO,CO_MESORREGIAO,...,QT_TUR_FUND_AF,QT_TUR_MED,QT_TUR_PROF,QT_TUR_PROF_TEC,QT_TUR_EJA,QT_TUR_EJA_FUND,QT_TUR_EJA_MED,QT_TUR_ESP,QT_TUR_ESP_CC,QT_TUR_ESP_CE
0,2012,Norte,1,Rondônia,RO,11,Alta Floresta D'Oeste,1100015,Leste Rondoniense,1102,...,1,0,0,0,0,0,0,0,0,0
1,2012,Norte,1,Rondônia,RO,11,Alta Floresta D'Oeste,1100015,Leste Rondoniense,1102,...,0,0,0,0,21,12,9,0,0,0
2,2012,Norte,1,Rondônia,RO,11,Alta Floresta D'Oeste,1100015,Leste Rondoniense,1102,...,2,0,0,0,0,0,0,0,0,0
3,2012,Norte,1,Rondônia,RO,11,Alta Floresta D'Oeste,1100015,Leste Rondoniense,1102,...,1,0,0,0,0,0,0,0,0,0
4,2012,Norte,1,Rondônia,RO,11,Alta Floresta D'Oeste,1100015,Leste Rondoniense,1102,...,0,0,0,0,0,0,0,0,0,0


In [9]:
df[df['CO_UF'] == 35]['NO_UF'].unique()

array(['São Paulo'], dtype=object)

In [10]:
def filtrar_sp(df: pd.DataFrame) -> pd.DataFrame:
    """
    Filtra o DataFrame para incluir apenas os dados do estado de São Paulo.
    """
    df_sp = df[df['CO_UF'] == 35].copy()  # 35 é o código do estado de São Paulo
    return df_sp

In [11]:
df_sp = filtrar_sp(df)
df_sp.head()

,NU_ANO_CENSO,NO_REGIAO,CO_REGIAO,NO_UF,SG_UF,CO_UF,NO_MUNICIPIO,CO_MUNICIPIO,NO_MESORREGIAO,CO_MESORREGIAO,...,QT_TUR_FUND_AF,QT_TUR_MED,QT_TUR_PROF,QT_TUR_PROF_TEC,QT_TUR_EJA,QT_TUR_EJA_FUND,QT_TUR_EJA_MED,QT_TUR_ESP,QT_TUR_ESP_CC,QT_TUR_ESP_CE
169851,2012,Sudeste,3,São Paulo,SP,35,Adamantina,3500105,Presidente Prudente,3508,...,11,10,0,0,1,0,1,9,9,0
169852,2012,Sudeste,3,São Paulo,SP,35,Adamantina,3500105,Presidente Prudente,3508,...,0,0,0,0,0,0,0,0,0,0
169853,2012,Sudeste,3,São Paulo,SP,35,Adamantina,3500105,Presidente Prudente,3508,...,15,7,0,0,2,1,1,10,10,0
169854,2012,Sudeste,3,São Paulo,SP,35,Adamantina,3500105,Presidente Prudente,3508,...,0,0,0,0,0,0,0,0,0,0
169855,2012,Sudeste,3,São Paulo,SP,35,Adamantina,3500105,Presidente Prudente,3508,...,0,9,17,17,0,0,0,0,0,0


In [12]:
def filtrar_escolas_estaduais(df: pd.DataFrame) -> pd.DataFrame:
    """
    Filtra o DataFrame para incluir apenas as escolas estaduais.
    """
    df_estaduais = df[df['TP_DEPENDENCIA'] == 2].copy()  # 2 é o código para escolas estaduais
    return df_estaduais

In [13]:
df_sp = filtrar_escolas_estaduais(df_sp)
df_sp.head()

,NU_ANO_CENSO,NO_REGIAO,CO_REGIAO,NO_UF,SG_UF,CO_UF,NO_MUNICIPIO,CO_MUNICIPIO,NO_MESORREGIAO,CO_MESORREGIAO,...,QT_TUR_FUND_AF,QT_TUR_MED,QT_TUR_PROF,QT_TUR_PROF_TEC,QT_TUR_EJA,QT_TUR_EJA_FUND,QT_TUR_EJA_MED,QT_TUR_ESP,QT_TUR_ESP_CC,QT_TUR_ESP_CE
169851,2012,Sudeste,3,São Paulo,SP,35,Adamantina,3500105,Presidente Prudente,3508,...,11,10,0,0,1,0,1,9,9,0
169852,2012,Sudeste,3,São Paulo,SP,35,Adamantina,3500105,Presidente Prudente,3508,...,0,0,0,0,0,0,0,0,0,0
169853,2012,Sudeste,3,São Paulo,SP,35,Adamantina,3500105,Presidente Prudente,3508,...,15,7,0,0,2,1,1,10,10,0
169854,2012,Sudeste,3,São Paulo,SP,35,Adamantina,3500105,Presidente Prudente,3508,...,0,0,0,0,0,0,0,0,0,0
169855,2012,Sudeste,3,São Paulo,SP,35,Adamantina,3500105,Presidente Prudente,3508,...,0,9,17,17,0,0,0,0,0,0


In [14]:
def filtrar_escolas_em_atividade(df: pd.DataFrame) -> pd.DataFrame:
    """
    Filtra o DataFrame para incluir apenas as escolas que estão ativas.
    """
    df_ativas = df[df['TP_SITUACAO_FUNCIONAMENTO'] == 1].copy()  # 1 é o código para escolas ativas
    return df_ativas

In [15]:
df_sp = filtrar_escolas_em_atividade(df_sp)
df_sp.head()

,NU_ANO_CENSO,NO_REGIAO,CO_REGIAO,NO_UF,SG_UF,CO_UF,NO_MUNICIPIO,CO_MUNICIPIO,NO_MESORREGIAO,CO_MESORREGIAO,...,QT_TUR_FUND_AF,QT_TUR_MED,QT_TUR_PROF,QT_TUR_PROF_TEC,QT_TUR_EJA,QT_TUR_EJA_FUND,QT_TUR_EJA_MED,QT_TUR_ESP,QT_TUR_ESP_CC,QT_TUR_ESP_CE
169851,2012,Sudeste,3,São Paulo,SP,35,Adamantina,3500105,Presidente Prudente,3508,...,11,10,0,0,1,0,1,9,9,0
169853,2012,Sudeste,3,São Paulo,SP,35,Adamantina,3500105,Presidente Prudente,3508,...,15,7,0,0,2,1,1,10,10,0
169855,2012,Sudeste,3,São Paulo,SP,35,Adamantina,3500105,Presidente Prudente,3508,...,0,9,17,17,0,0,0,0,0,0
169856,2012,Sudeste,3,São Paulo,SP,35,Adamantina,3500105,Presidente Prudente,3508,...,0,3,12,12,0,0,0,1,1,0
169857,2012,Sudeste,3,São Paulo,SP,35,Adamantina,3500105,Presidente Prudente,3508,...,22,12,0,0,0,0,0,6,6,0


In [16]:
matriculas_basica_cor = [
    'QT_MAT_BAS_ND',
    'QT_MAT_BAS_BRANCA',
    'QT_MAT_BAS_PRETA',
    'QT_MAT_BAS_PARDA',
    'QT_MAT_BAS_AMARELA',
    'QT_MAT_BAS_INDIGENA',

]

In [17]:
matriculas_integrais = [
    'QT_MAT_INF_INT',
    'QT_MAT_INF_CRE_INT',
    'QT_MAT_INF_PRE_INT',
    'QT_MAT_FUND_INT',
    'QT_MAT_FUND_AI_INT',
    'QT_MAT_FUND_AF_INT', #esse é do PEI
    'QT_MAT_MED_INT', # esse também

]

In [18]:
totais = [
    'QT_TUR_BAS', #quantidade de turmas da educação básica
    'QT_MAT_BAS', #quantidade de matrículas da educação básica
]

In [19]:
indices = [
    'NU_ANO_CENSO', #ano do censo
    'SG_UF', #sigla da unidade federativa
    'NO_MUNICIPIO',
    'CO_MUNICIPIO', #código do município
    'CO_ENTIDADE', #código da escola
    'NO_ENTIDADE', #nome da escola
]

In [20]:
cols_interesse = indices + totais + matriculas_basica_cor + matriculas_integrais

In [21]:
df_sp[cols_interesse].head()

,NU_ANO_CENSO,SG_UF,NO_MUNICIPIO,CO_MUNICIPIO,CO_ENTIDADE,NO_ENTIDADE,QT_TUR_BAS,QT_MAT_BAS,QT_MAT_BAS_ND,QT_MAT_BAS_BRANCA,...,QT_MAT_BAS_PARDA,QT_MAT_BAS_AMARELA,QT_MAT_BAS_INDIGENA,QT_MAT_INF_INT,QT_MAT_INF_CRE_INT,QT_MAT_INF_PRE_INT,QT_MAT_FUND_INT,QT_MAT_FUND_AI_INT,QT_MAT_FUND_AF_INT,QT_MAT_MED_INT
169851,2012,SP,Adamantina,3500105,35030806,HELEN KELLER,22,654,87,355,...,199,1,1,0,0,0,0,0,0,0
169853,2012,SP,Adamantina,3500105,35031045,DURVALINO GRION PROF,24,827,146,439,...,224,5,1,0,0,0,0,0,0,0
169855,2012,SP,Adamantina,3500105,35031082,EUDECIO LUIZ VICENTE PROF ETE,26,883,163,563,...,130,21,1,0,0,0,0,0,0,0
169856,2012,SP,Adamantina,3500105,35031100,HERVAL BELLUSCI ENGENHEIRO ETE,12,275,68,140,...,62,1,0,0,0,0,0,0,0,68
169857,2012,SP,Adamantina,3500105,35031112,FLEURIDES CAVALLINI MENECHINO PROFA,34,1041,84,758,...,180,11,3,0,0,0,0,0,0,0


In [22]:
from typing import Optional
import requests
import os
from tqdm import tqdm
from config import RANGE_ANOS_ESTUDO, DATA_FOLDER

class ExtractCensoEscolar:

    URL_DOWNLOAD = 'https://download.inep.gov.br/dados_abertos/microdados_censo_escolar_{ano}.zip'

    def __init__(self, range_anos:Optional[list[int]]=None):
         
         self.range_anos = range_anos or RANGE_ANOS_ESTUDO

    def solve_file_microdados_censo(self, ano:int) -> str:

            fname = f'microdados_censo_{ano}.zip'
            if not os.path.exists(DATA_FOLDER):
                os.mkdir(DATA_FOLDER)
            fpath = os.path.join(DATA_FOLDER, fname)

            return fpath


    def download_microdados_censo(self, ano:int)->str:

        link_download_microdados_censo = self.URL_DOWNLOAD.format(ano=ano)
        fpath = self.solve_file_microdados_censo(ano)
        with requests.get(link_download_microdados_censo, stream=True) as r:
            r.raise_for_status()
            with open(fpath, 'wb') as f:
                content = r.content
                f.write(content)
                
        return fpath
    
    def pipeline(self)->list[str]:

        files = []
        print('Iniciando o download dos microdados do censo escolar...')
        ja_baixados = []
        baixados = []
        for ano in tqdm(self.range_anos):
            file = self.solve_file_microdados_censo(ano)
            if not os.path.exists(file):
                baixados.append(ano)
                self.download_microdados_censo(ano)
            else:
                ja_baixados.append(ano)

        print(f'Microdados do censo escolar já baixados previamente para os anos: {ja_baixados}')
        print(f'Microdados do censo escolar baixados agora para os anos: {baixados}')
        return files

    def __call__(self)->list[str]:
        """
        Executa o pipeline de download dos microdados do censo escolar.
        """
        return self.pipeline()


In [23]:
from utils.save_csv import save_csv
from typing import Optional

class ETLMicrodadosCensoEscolar:


    def __init__(self, range_anos:Optional[list[int]]=None) -> None:

        self.range_anos = range_anos or RANGE_ANOS_ESTUDO
        self.extract_censo = ExtractCensoEscolar(self.range_anos)

    @property
    def cols_mapper(self) -> dict[str, str]:
        """
        Mapeia as colunas de interesse para os nomes originais.
        """
        indices = {
            'NU_ANO_CENSO' : 'ano_censo', 
            'SG_UF' : 'sigla_uf', 
            'NO_MUNICIPIO' : 'nome_municipio',
            'CO_MUNICIPIO' : 'codigo_municipio', 
            'CO_ENTIDADE' : 'codigo_escola', 
            'NO_ENTIDADE' : 'nome_escola', 
        }
        totais = {
            'QT_TUR_BAS' : 'qtd_turmas_ed_basica', #quantidade de turmas da educação básica
            'QT_MAT_BAS' : 'qtd_matr_ed_basica', #quantidade de matrículas da educação básica
        }
        matriculas_basica_cor = {
            'QT_MAT_BAS_ND' : 'qtd_matr_ed_basica_nao_declarada',
            'QT_MAT_BAS_BRANCA' : 'qtd_matr_ed_basica_branca',
            'QT_MAT_BAS_PRETA' : 'qtd_matr_ed_basica_preta',
            'QT_MAT_BAS_PARDA' : 'qtd_matr_ed_basica_parda',
            'QT_MAT_BAS_AMARELA' : 'qtd_matr_ed_basica_amarela',
            'QT_MAT_BAS_INDIGENA' : 'qtd_matr_ed_basica_indigena',
        }
        matriculas_integrais = {
            'QT_MAT_INF_INT' : 'qtd_matriculas_infantil_integral',
            'QT_MAT_INF_CRE_INT' : 'qtd_matriculas_infantil_creche_integral',
            'QT_MAT_INF_PRE_INT' : 'qtd_matriculas_infantil_pre_escolar_integral',
            'QT_MAT_FUND_INT' : 'qtd_matriculas_fundamental_integral',
            'QT_MAT_FUND_AI_INT' : 'qtd_matriculas_fundamental_anos_iniciais_integral',
            'QT_MAT_FUND_AF_INT' : 'qtd_matriculas_fundamental_anos_finais_integral', #esse é do PEI
            'QT_MAT_MED_INT' : 'qtd_matriculas_medio_integral', # esse também
        }

        return {**indices, **totais, **matriculas_basica_cor, **matriculas_integrais}

    @property
    def cols_interesse(self) -> list[str]:
        return list(self.cols_mapper.keys())

    def solve_data_folder_within_zip(self, ano:int) -> str:

        folder_map = {
            2022: f'Microdados do Censo Escolar da Educaç╞o Básica {ano}',
            2023: f'microdados_censo_escolar_{ano}',
            2024 : f'microdados_censo_escolar_{ano}'
        }

        folder = folder_map.get(ano, f'microdados_ed_basica_{ano}')
        return folder
    
    def solve_csv_data_path_within_zip(self, ano:int) -> str:
    
        fname: str = f'microdados_ed_basica_{ano}.csv'
        
        folder = self.solve_data_folder_within_zip(ano)
        
        path: str = os.path.join(folder, 'dados', fname)

        return path

    def read_data(self, ano:int)->pd.DataFrame:

        fpath = self.extract_censo.solve_file_microdados_censo(ano)
        if not os.path.exists(fpath):
            raise FileNotFoundError(f'Arquivo {fpath} não encontrado. Baixe os dados primeiro.')

        zip_file_path = self.solve_csv_data_path_within_zip(ano)
        with ZipFile(fpath, 'r') as zip_ref:
            if zip_file_path not in zip_ref.namelist():
                #por alguma razão, o arquivo pode estar com extensão .CSV em vez de .csv
                zip_file_path = zip_file_path.replace('.csv', '.CSV')
                #caso nao encontre mesmo assim, levanta o erro
                if zip_file_path not in zip_ref.namelist():
                    raise FileNotFoundError(f'Arquivo {zip_file_path} não encontrado no zip {fpath}.')
            with zip_ref.open(zip_file_path) as file:
                df = pd.read_csv(file, sep=';', encoding='latin1', low_memory=False)
            return df
        
    def filtrar_sp(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Filtra o DataFrame para incluir apenas os dados do estado de São Paulo.
        """
        df = df[df['CO_UF'] == 35].copy()  # 35 é o código do estado de São Paulo
        return df
    
    def filtrar_escolas_estaduais(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Filtra o DataFrame para incluir apenas as escolas estaduais.
        """
        # 2 é o código para escolas estaduais
        df = df[df['TP_DEPENDENCIA'] == 2].copy()
        return df
    
    def filtrar_escolas_em_atividade(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Filtra o DataFrame para incluir apenas as escolas que estão ativas.
        """
        # 1 é o código para escolas ativas
        df = df[df['TP_SITUACAO_FUNCIONAMENTO'] == 1].copy()
        return df
    
    def filtrar_cols_interesse(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Filtra o DataFrame para incluir apenas as colunas de interesse.
        """
        df = df[self.cols_interesse].copy()
        return df
    
    def assert_cols_match(self, df: pd.DataFrame) -> None:
        """
        Verifica se as colunas do DataFrame correspondem às colunas de interesse.
        """
        if not (df.columns == self.cols_interesse).all():
            raise ValueError(f'As colunas do DataFrame não correspondem às colunas de interesse.'
                            f'Esperadas: {self.cols_interesse}, encontradas: {df.columns.tolist()}')
        
    def map_cols(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Mapeia as colunas do DataFrame para os nomes originais.
        """
        
        self.assert_cols_match(df)
        df = df.rename(columns=self.cols_mapper)
        return df

    def pipeline(self) -> pd.DataFrame:
        """
        Executa o pipeline de extração, transformação e carregamento dos microdados do censo escolar.
        """

        print('Iniciando o pipeline de ETL dos microdados do censo escolar...')
        self.extract_censo.pipeline()  # Garante que os dados foram baixados
        df_final = pd.DataFrame()
        print('Lendo e processando os dados...')
        for ano in tqdm(self.range_anos):
            df = self.read_data(ano)
            df = self.filtrar_sp(df)
            df = self.filtrar_escolas_estaduais(df)
            df = self.filtrar_escolas_em_atividade(df)
            df = self.filtrar_cols_interesse(df)
            df_final = pd.concat([df_final, df], ignore_index=True)

        df_final = self.map_cols(df_final)
        print('Pipeline de ETL concluído.')

        return df_final
    

    @property
    def save_file_path(self) -> str:
        """
        Retorna o caminho do arquivo CSV onde os dados serão salvos.
        """
        return 'microdados_censo_limpo.csv'
    
    def __call__(self, save_csv_file=True) -> pd.DataFrame:
        """
        Executa o pipeline de extração, transformação e carregamento dos microdados do censo escolar.
        """

        fpath = self.save_file_path
        if os.path.exists(os.path.join(DATA_FOLDER, fpath)):
            print(f'Arquivo com dados limpos já existe. Carregando os dados do arquivo...')
            df = pd.read_csv(os.path.join(DATA_FOLDER, fpath))
            return df

        df = self.pipeline()
        if save_csv_file:
            save_csv(df, 'microdados_censo_limpo.csv')
        return df
            

In [24]:
pipeline_censo = ETLMicrodadosCensoEscolar()
df_censo = pipeline_censo()

Iniciando o pipeline de ETL dos microdados do censo escolar...
Iniciando o download dos microdados do censo escolar...


100%|██████████| 13/13 [00:00<00:00, 66172.27it/s]


Microdados do censo escolar já baixados previamente para os anos: [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
Microdados do censo escolar baixados agora para os anos: []
Lendo e processando os dados...


100%|██████████| 13/13 [01:02<00:00,  4.81s/it]


Pipeline de ETL concluído.
Data saved to data/microdados_censo_limpo.csv


In [25]:
df_censo.head()

,ano_censo,sigla_uf,nome_municipio,codigo_municipio,codigo_escola,nome_escola,qtd_turmas_ed_basica,qtd_matr_ed_basica,qtd_matr_ed_basica_nao_declarada,qtd_matr_ed_basica_branca,...,qtd_matr_ed_basica_parda,qtd_matr_ed_basica_amarela,qtd_matr_ed_basica_indigena,qtd_matriculas_infantil_integral,qtd_matriculas_infantil_creche_integral,qtd_matriculas_infantil_pre_escolar_integral,qtd_matriculas_fundamental_integral,qtd_matriculas_fundamental_anos_iniciais_integral,qtd_matriculas_fundamental_anos_finais_integral,qtd_matriculas_medio_integral
0,2012,SP,Adamantina,3500105,35030806,HELEN KELLER,22.0,654.0,87.0,355.0,...,199.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2012,SP,Adamantina,3500105,35031045,DURVALINO GRION PROF,24.0,827.0,146.0,439.0,...,224.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2012,SP,Adamantina,3500105,35031082,EUDECIO LUIZ VICENTE PROF ETE,26.0,883.0,163.0,563.0,...,130.0,21.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2012,SP,Adamantina,3500105,35031100,HERVAL BELLUSCI ENGENHEIRO ETE,12.0,275.0,68.0,140.0,...,62.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,68.0
4,2012,SP,Adamantina,3500105,35031112,FLEURIDES CAVALLINI MENECHINO PROFA,34.0,1041.0,84.0,758.0,...,180.0,11.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
len(df_censo)

75928